In [4]:
from manim import *
import numpy as n

In [11]:
%%manim -ql --disable_caching --flush_cache -v WARNING Fabrik

from manim import *
import numpy as np

class Fabrik(Scene):
    def construct(self):
        font = "Avenir"
        title = Text("FABRIK 前向传递原理", font=font).scale(0.8).to_edge(UP, buff=0.5)
        self.play(Write(title))

        # 创建初始链条，放在左侧
        dots = VGroup(*[Dot() for _ in range(4)])
        dots.arrange(RIGHT, buff=1)
        dots.next_to(title, DOWN, buff=1.5)  # 增加与标题的距离
        
        lines = VGroup(*[Line(dots[i], dots[i+1]) for i in range(len(dots)-1)])
        chain = VGroup(dots, lines)

        labels = VGroup(*[
            Text(f"关节{i}", font=font).scale(0.4).next_to(dot, DOWN, buff=0.1)
            for i, dot in enumerate(dots)
        ])

        self.play(Create(chain), Write(labels))

        # 创建目标点，放在右上方
        target = Dot(color=YELLOW).move_to(dots[-1].get_center() + RIGHT * 2 + UP * 1.5)
        target_label = Text("目标点", font=font).scale(0.4).next_to(target, UP, buff=0.1)
        self.play(Create(target), Write(target_label))

        # 保存原始位置
        original_positions = [dot.get_center() for dot in dots]
        
        # 前向传递说明文字
        forward_explanation = VGroup(
            Text("1. 末端关节直接移动到目标点", font=font),
            Text("2. 其他关节在保持连杆长度的情况下", font=font),
            Text("3. 沿圆弧移动到最近的可行位置", font=font)
        ).arrange(DOWN, aligned_edge=LEFT).scale(0.4).to_edge(RIGHT)
        
        self.play(Write(forward_explanation))

        # 前向传递动画
        self.play(dots[-1].animate.move_to(target.get_center()))
        
        for i in range(len(dots)-2, -1, -1):
            old_pos = dots[i].get_center()
            next_dot = dots[i+1]
            
            # 1. 显示当前关节到下一个关节的连线
            current_line = Line(old_pos, next_dot.get_center(), color=BLUE)
            self.play(Create(current_line))
            
            # 2. 显示固定长度的圆弧
            original_length = np.linalg.norm(original_positions[i+1] - original_positions[i])
            circle = Circle(radius=original_length, color=RED_A)
            circle.move_to(next_dot.get_center())
            self.play(Create(circle))
            
            # 3. 计算新位置
            direction = old_pos - next_dot.get_center()
            direction = direction / np.linalg.norm(direction)
            new_pos = next_dot.get_center() + direction * original_length
            
            # 4. 显示新位置和移动过程
            arc_arrow = CurvedArrow(
                start_point=old_pos,
                end_point=new_pos,
                angle=-TAU/8,
                color=YELLOW
            )
            self.play(Create(arc_arrow))
            
            # 5. 移动关节到新位置
            self.play(
                dots[i].animate.move_to(new_pos),
                lines[i].animate.put_start_and_end_on(new_pos, dots[i+1].get_center()),
            )
            
            # 6. 清除辅助图形
            self.play(
                FadeOut(current_line),
                FadeOut(circle),
                FadeOut(arc_arrow)
            )

        # 清理舞台，准备后向传递
        self.play(
            FadeOut(forward_explanation),
            FadeOut(target_label),
            FadeOut(title)
        )

        # 添加后向传递的标题
        backward_title = Text("FABRIK 后向传递原理", font=font).scale(0.8).to_edge(UP, buff=0.5)
        self.play(Write(backward_title))

        # 显示基座位置
        base = Dot(color=GREEN).move_to(original_positions[0])
        base_label = Text("基座", font=font).scale(0.4).next_to(base, DOWN, buff=0.1)
        self.play(Create(base), Write(base_label))

        # 后向传递说明文字
        backward_explanation = VGroup(
            Text("1. 基座关节回到原始位置", font=font),
            Text("2. 其他关节保持连杆长度", font=font),
            Text("3. 依次调整到最近可行位置", font=font)
        ).arrange(DOWN, aligned_edge=LEFT).scale(0.4).to_edge(RIGHT)
        
        self.play(Write(backward_explanation))

        # 后向传递动画
        self.play(dots[0].animate.move_to(base.get_center()))
        
        for i in range(1, len(dots)):
            old_pos = dots[i].get_center()
            prev_dot = dots[i-1]
            
            # 1. 显示当前关节到前一个关节的连线
            current_line = Line(old_pos, prev_dot.get_center(), color=BLUE)
            self.play(Create(current_line))
            
            # 2. 显示固定长度的圆弧
            original_length = np.linalg.norm(original_positions[i] - original_positions[i-1])
            circle = Circle(radius=original_length, color=RED_A)
            circle.move_to(prev_dot.get_center())
            self.play(Create(circle))
            
            # 3. 计算新位置
            direction = old_pos - prev_dot.get_center()
            direction = direction / np.linalg.norm(direction)
            new_pos = prev_dot.get_center() + direction * original_length
            
            # 4. 显示新位置和移动过程
            arc_arrow = CurvedArrow(
                start_point=old_pos,
                end_point=new_pos,
                angle=-TAU/8,
                color=YELLOW
            )
            self.play(Create(arc_arrow))
            
            # 5. 移动关节到新位置
            self.play(
                dots[i].animate.move_to(new_pos),
                lines[i-1].animate.put_start_and_end_on(prev_dot.get_center(), new_pos),
            )
            
            # 6. 清除辅助图形
            self.play(
                FadeOut(current_line),
                FadeOut(circle),
                FadeOut(arc_arrow)
            )

        # 最终总结
        self.play(
            FadeOut(backward_explanation),
            FadeOut(backward_title),
            FadeOut(base_label),
            FadeOut(base)
        )

        final_title = Text("FABRIK 算法原理", font=font).scale(0.8).to_edge(UP, buff=0.5)
        summary = Text(
            "通过反复的前向和后向传递，最终收敛到目标位置",
            font=font
        ).scale(0.4).to_edge(DOWN)
        
        self.play(
            Write(final_title),
            Write(summary)
        )
        self.wait(2)


Manim Community v0.18.1